### Current Hyperparameter settings:
* gamma = 2550
* lambda = 10

In [1]:
gamma = 2550
lambdaz = 10

In [2]:
import numpy as np
import random
import scipy.io
import scipy.ndimage
import os
import gc
import random
from PIL import Image
import cv2
import h5py
import random

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as T
import torch.nn.functional as Fn
from torch.autograd import Variable
from torch.utils.data import DataLoader, TensorDataset

from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
%matplotlib inline

/usr/local/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [ ]:
X_A_dir = '/ShanghaiTech_Crowd_Counting_Dataset/part_A_final/train_data/images/'
Y_A_dir = '/ShanghaiTech_Crowd_Counting_Dataset/part_A_final/train_data/ground_truth/'

X_A_tdir = '/ShanghaiTech_Crowd_Counting_Dataset/part_A_final/test_data/images/'
Y_A_tdir = '/ShanghaiTech_Crowd_Counting_Dataset/part_A_final/test_data/ground_truth/'

X_B_dir = '/ShanghaiTech_Crowd_Counting_Dataset/part_B_final/train_data/images/'
Y_B_dir = '/ShanghaiTech_Crowd_Counting_Dataset/part_B_final/train_data/ground_truth/'

X_B_tdir = '/ShanghaiTech_Crowd_Counting_Dataset/part_B_final/test_data/images/'
Y_B_tdir = '/ShanghaiTech_Crowd_Counting_Dataset/part_B_final/test_data/ground_truth/'

images_A = [X_A_dir + i for i in os.listdir(X_A_dir)]
images_At = [X_A_tdir + i for i in os.listdir(X_A_tdir)]
images_B = [X_B_dir + i for i in os.listdir(X_B_dir)]
images_Bt = [X_B_tdir + i for i in os.listdir(X_B_tdir)]

gt_A = [Y_A_dir + i for i in os.listdir(Y_A_dir)]
gt_At = [Y_A_tdir + i for i in os.listdir(Y_A_tdir)]
gt_B = [Y_B_dir + i for i in os.listdir(Y_B_dir)]
gt_Bt = [Y_B_tdir + i for i in os.listdir(Y_B_tdir)]

images_train = images_A + images_B
images_test = images_At + images_Bt

gt_train = gt_A + gt_B
gt_test = gt_At + gt_Bt

images_train.sort()
images_test.sort()
gt_train.sort()
gt_test.sort()

In [ ]:
def rgb2gray(rgb):                                                      #convert RGB images to grayscale
    return np.dot(rgb[...,:3], [0.299, 0.587, 0.114])

X_train = []
X_test_A = []
X_test_B = []

def generate_data():
    
    for image in images_train:
        X_train.append(rgb2gray(cv2.resize(cv2.imread(image), (256, 256))))
        
    for i in xrange(182):
        X_test_A.append(rgb2gray(cv2.resize(cv2.imread(images_test[i]), (256, 256))))
        
    for i in xrange(182, 498):
        X_test_B.append(rgb2gray(cv2.resize(cv2.imread(images_test[i]), (256, 256))))
        
    return X_train, X_test_A, X_test_B

X_train, X_test_A, X_test_B = generate_data()

print 'Shape of a sample training image: {}'.format(X_train[0].shape)

In [ ]:
Y_test_A = np.asarray(np.load('/ShanghaiDensities256/densities_test_A.npy'))
Y_test_B = np.asarray(np.load('/ShanghaiDensities256/densities_test_B.npy'))
Y_train = np.asarray(np.load('/ShanghaiDensities256/densities_train.npy'))

print 'Sizes\nTraining: {} (A: {}, B: {})\nTest A: {}\nTest B: {}'.format(len(X_train), len(images_A), len(images_B), len(X_test_A), len(X_test_B))

In [ ]:
class UnetSkipConnectionBlock(nn.Module):
    def __init__(self, outer_nc, inner_nc, norm_layer=nn.BatchNorm2d, input_nc=None, submodule=None, outermost=False, innermost=False):
        super(UnetSkipConnectionBlock, self).__init__()
        
        self.outermost = outermost
        if input_nc == None:
            input_nc = outer_nc
            
        use_bias = norm_layer == nn.InstanceNorm2d
        
        downconv = nn.Conv2d(input_nc, inner_nc, kernel_size=4, stride=2, padding=1, bias=use_bias)
        downrelu = nn.LeakyReLU(0.2, True)
        downnorm = norm_layer(inner_nc)
        uprelu = nn.ReLU(True)
        upnorm = norm_layer(outer_nc)
        
        if outermost:
            upconv = nn.ConvTranspose2d(inner_nc * 2, outer_nc, kernel_size=4, stride=2, padding=1)
            down = [downconv]
            up = [uprelu, upconv, nn.ReLU()]
            model = down + [submodule] + up
            
        elif innermost:
            upconv = nn.ConvTranspose2d(inner_nc, outer_nc, kernel_size=4, stride=2, padding=1, bias=use_bias)
            down = [downrelu, downconv]
            up = [uprelu, upconv, upnorm]
            model = down + up
            
        else:
            upconv = nn.ConvTranspose2d(inner_nc * 2, outer_nc, kernel_size=4, stride=2, padding=1, bias=use_bias)
            down = [downrelu, downconv, downnorm]
            up = [uprelu, upconv, upnorm]
            
            model = down + [submodule] + up + [nn.Dropout(0.5)]
            
        self.model = nn.Sequential(*model)
        
    def forward(self, input):
        if self.outermost:
            return self.model(input)
        
        else:
            return torch.cat([input, self.model(input)], 1)

In [ ]:
class UnetGenerator(nn.Module):
    def __init__(self, input_nc, output_nc, ngf=64, norm_layer=nn.BatchNorm2d, num_downs=4):
        super(UnetGenerator, self).__init__()
                
        unet_block = UnetSkipConnectionBlock(ngf * 8, ngf * 8, input_nc=None, norm_layer=norm_layer, innermost=True)
        
        for i in xrange(num_downs - 1):
            unet_block = UnetSkipConnectionBlock(ngf * 8, ngf * 8, input_nc=None, submodule=unet_block, norm_layer=norm_layer)
            
        unet_block = UnetSkipConnectionBlock(ngf * 4, ngf * 8, input_nc=None, submodule=unet_block, norm_layer=norm_layer)
        unet_block = UnetSkipConnectionBlock(ngf * 2, ngf * 4, input_nc=None, submodule=unet_block, norm_layer=norm_layer)
        unet_block = UnetSkipConnectionBlock(ngf, ngf * 2, input_nc=None, submodule=unet_block, norm_layer=norm_layer)
        unet_block = UnetSkipConnectionBlock(output_nc, ngf, input_nc=input_nc, submodule=unet_block, outermost=True, norm_layer=norm_layer)
        
        self.model = unet_block
        
    def forward(self, input):
        return self.model(input)

In [ ]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)

In [ ]:
class Discriminator(nn.Module):
    def __init__(self, input_nc):
        super(Discriminator, self).__init__()
        
        layers = [nn.Conv2d(1, 64, kernel_size=4, stride=2)]
        inner_c=64

        for _ in xrange(4):
            layers += [nn.Conv2d(inner_c, inner_c * 2, kernel_size=4, stride=2), nn.LeakyReLU(0.2, True), nn.BatchNorm2d(inner_c * 2)]
            inner_c = inner_c * 2
            
        layers += [nn.Conv2d(inner_c, 1, kernel_size=6, stride=1), nn.LeakyReLU(0.2, True)]
        self.model = nn.Sequential(*layers)
    
    def forward(self, input):
        output = self.model(input).view(1, -1)
        return Fn.sigmoid(output)

In [ ]:
for i in xrange(300):
    Y_train[i] = Y_train[i] * gamma

In [ ]:
X_train_tensor = torch.from_numpy(np.asarray(X_train)).unsqueeze(3)
Y_train_tensor = torch.from_numpy(np.asarray(Y_train)).unsqueeze(3)

X_train_tensor = X_train_tensor[:300].permute(0, 3, 1, 2)          #Only for part A first
Y_train_tensor = Y_train_tensor[:300].permute(0, 3, 1, 2)

train_data = TensorDataset(X_train_tensor, Y_train_tensor)
train_loader = DataLoader(train_data, batch_size=1, pin_memory = False)

print X_train_tensor.size(), Y_train_tensor.size()

In [ ]:
dtype = torch.FloatTensor

if torch.cuda.is_available():
    dtype = torch.cuda.FloatTensor
print dtype
    
G = UnetGenerator(1, 1, norm_layer=nn.BatchNorm2d, num_downs=4).type(dtype)         #apply weights_init for both
D = Discriminator(1).type(dtype)

G.apply(weights_init)
D.apply(weights_init)

#G.load_state_dict(torch.load('G.pt'))
#D.load_state_dict(torch.load('D.pt'))

G_optim = optim.Adam(G.parameters(), lr=0.0002, betas=(0.5, 0.999))  
D_optim = optim.Adam(D.parameters(), lr=0.0002, betas=(0.5, 0.999))

#G_optim.load_state_dict(torch.load('/model/G_optim.pt'))
#D_optim.load_state_dict(torch.load('/model/D_optim.pt'))

In [ ]:
def test_on_test(mode = 'A', number = 105, evalz=False):
    
    if evalz == True:
        G.eval()
    
    if mode == 'A':
        img = X_test_A[number]
        mode = Y_test_A
    elif mode == 'B':
        img = X_test_B[number]
        mode = Y_test_B
    else:
        print 'Invalid mode. Use only A or B'
    
    print 'Creating tensor from image...'
    img_v = Variable(torch.from_numpy(img).unsqueeze(0).unsqueeze(1)).type(dtype)
    print 'Tensor of shape: {} created'.format(img_v.size())
    
    #Pass through G
    density_map = G(img_v).view(256, 256).data.cpu().numpy()
    density_map = density_map / gamma
    
    plt.imshow(img)
    plt.title('Image [{}]'.format(np.sum(mode[number])))
    plt.figure()
    plt.title('Density map generated [{}]'.format(np.sum(density_map)))
    plt.imshow(density_map)
    plt.figure()
    plt.title('Ground truth [{}]'.format(np.sum(mode[number])))
    plt.imshow(mode[number])

In [ ]:
loss_L1 = nn.L1Loss().type(dtype)

def train(models, optimizers, loss_fn, epochs, evalz = False):
    
    gen, disc = models
    gen_optim, disc_optim = optimizers
    
    for epoch in xrange(epochs):
        gen.train()
        disc.train()
        
        print 'Epoch {} started'.format(epoch + 1)
        
        for i, (X, Y) in enumerate(train_loader):
            X = Variable(X).type(dtype)       #first train discriminator
            Y = Variable(Y).type(dtype)
            
            Y_fake = gen(X)
            true_ = Variable(torch.ones(len(X), 1)).type(dtype)
            fake_ = Variable(torch.zeros(len(X), 1)).type(dtype)
            
            for _ in xrange(2):
                scores_fake = disc(Y_fake)
                scores_real = disc(Y)

                loss1 = loss_fn(scores_fake, fake_)
                loss2 = loss_fn(scores_real, true_)
                loss_d = (loss1 + loss2) * 0.5 

                disc_optim.zero_grad()
                loss_d.backward(retain_graph=True)
                disc_optim.step()
            
            if i % 100 == 0:
                    print 'Discriminator loss after {} batches: {}'.format(i, loss_d.data[0])
            
            #train generator
            
            loss_g = loss_fn(scores_fake, true_) + lambdaz * loss_L1(Y_fake, Y)
            
            if i % 100 == 0:
                print 'Generator loss after {} batches: {}'.format(i, loss_g.data[0])
            
            gen_optim.zero_grad()
            loss_g.backward()
            gen_optim.step()
            
        print 'Epoch ended'
        
        if epoch % 2 == 0:
            n = random.randint(0, 181)
            test_on_test(mode='A', number=n, evalz=evalz)
            
        torch.save(G.state_dict(), 'G.pt')
        torch.save(D.state_dict(), 'D.pt')
        torch.save(G_optim.state_dict(), 'G_optim.pt')
        torch.save(D_optim.state_dict(), 'D_optim.pt')

In [ ]:
loss_fn = nn.BCELoss().type(dtype)
models = [G, D]
optimizers = [G_optim, D_optim]

train(models, optimizers, loss_fn, 10)

In [ ]:
train(models, optimizers, loss_fn, 10)

In [ ]:
train(models, optimizers, loss_fn, 10)

In [ ]:
train(models, optimizers, loss_fn, 10, evalz = True)